In [70]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import time
import json

In [71]:
options =uc.ChromeOptions()
driver = uc.Chrome(options)

In [72]:
driver.get('https://www.gelbeseiten.de/suche/steuerberatung/essen?umkreis=50000')
time.sleep(3)

In [73]:
driver.find_element(By.XPATH, '//span[@id="cmpwelcomebtnyes"]/a').click()

In [7]:
total = 500
loadmorebutton = driver.find_element(By.XPATH, '//a[@id="mod-LoadMore--button"]')

while len(driver.find_elements(By.XPATH, '//div[@id="teilnehmer_block"]/div/article'))!= total:
    try:
        driver.execute_script("arguments[0].scrollIntoView(false);", loadmorebutton)
        loadmorebutton.click()
        print( len(driver.find_elements(By.XPATH, '//div[@id="teilnehmer_block"]/div/article')))
        time.sleep(3)
    except:
        loadmore = False


50
60
70
80
90
100
110
120
130
140
150
160
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500


In [60]:
driver.find_element(By.XPATH, './/div[@id="email_versenden"]').get_attribute("data-link").removeprefix('mailto:').removesuffix("?subject=Anfrage über Gelbe Seiten")

'steuerberatung@kueppers-tonner.de'

In [74]:
sample = driver.find_element(By.XPATH, '//*[@id="gs_treffer"]/article')
sample.find_element(By.XPATH, ".//a").get_attribute('href')

'https://www.gelbeseiten.de/gsbiz/7743b3ea-058b-49ed-9dce-52ce7f3a6e1d'

In [80]:
data =[]
profiles = driver.find_elements(By.XPATH, '//*[@id="gs_treffer"]/article')

for profile in profiles:
    driver.execute_script("arguments[0].scrollIntoView();", profile)
    time.sleep(2)
    
    distance = profile.find_element(By.XPATH, './/span[@class="mod-AdresseKompakt__entfernung"]').text
    print(distance)

    link = profile.find_element(By.XPATH, ".//a").get_attribute('href')
    page = driver.execute_script("window.open('{}', '_blank');".format(link))
    driver.switch_to.window(driver.window_handles[1])
    '''
    data.append({
        'Name': driver.find_element(By.XPATH, '//div[@class="mod-TeilnehmerKopf__Name-wrapper"]/h1').text,
        'TaxAdvice': driver.find_element(By.XPATH, '//div[@class="mod-TeilnehmerKopf__branchen"]/span[3]').text.replace('TAX ADVICE: ', ''),
        "Email": driver.find_element(By.XPATH, '//div[@id="email_versenden"]').get_attribute("data-link").removeprefix('mailto:').removesuffix("?subject=Anfrage über Gelbe Seiten"),
        'Location':(driver.find_element(By.XPATH, '//address[@class="mod-TeilnehmerKopf__adresse"]/div/span').text) + ' ' +
                (driver.find_element(By.XPATH, '//address[@class="mod-TeilnehmerKopf__adresse"]/div/span[2]').text) + ' ' +
                (driver.find_element(By.XPATH, '//address[@class="mod-TeilnehmerKopf__adresse"]/div/span[3]').text),
        'Distance': distance,            
        'Phone': driver.find_element(By.XPATH, '//span[@data-role="telefonnummer"]/a/span').text,
        'Website': driver.find_element(By.XPATH, '//a[@data-wipe-realview="detailseite_webadresse"]').get_attribute("href"),
    })'''

    time.sleep(2)
    driver.close()

    driver.switch_to.window(driver.window_handles[0])
    time.sleep(2)

#with open('german.json', 'w') as g:
    #json.dump(data, g)

driver.quit()


1,3 km


IndexError: list index out of range